In [1]:
# scrape webpage

#https://letslearnabout.net/tutorial/scrapy-tutorial/python-scrapy-tutorial-for-beginners-01-creating-your-first-spider/
#https://letslearnabout.net/tutorial/python-scrapy-tutorial-for-beginners-03-how-to-go-to-the-next-page/

"""
to do:

fields:
Достигната цена (лв.)
Отстранени участници
Търг за местни фирми (да/не)

Обем дървесина (м3) == Общо??
Крайна цена лв./м3  = Договорена цена (лв.)/Общо

Договорена цена (лв.) == Първа цена?
Крайна цена лв./м3  = Договорена цена (лв.)/Общо

Процентна разлика постигната/ начална цена (%)


Забележки: 
без плик с цена
без ценови предложения
герб/ дпс?


excel sheets:
добив до склад/ продажба корен-сечище



"""

import os
import sys
import scrapy
from scrapy.crawler import CrawlerProcess
from scrapy.utils.log import configure_logging
import logging
from imp import reload
reload(logging)

filename = f'{os.path.join(os.getcwd())}/logging.txt'
logging.basicConfig(filename=filename,
                    filemode='a',
#                     encoding='utf-8',
                    format='%(asctime)s,%(msecs)d %(name)s %(levelname)s %(message)s',
                    datefmt='%H:%M:%S',
                    level=logging.DEBUG)

import re
from datetime import datetime 
from datetime import date

import json
import pandas as pd
import numpy as np

import PyPDF2
import requests
from nltk import sent_tokenize
sys.path.insert(0, "/Users/bilyanavencheva/Desktop/Side projects/DataForGood/wood")
from functions import transform_clean_data, get_latest_file



"""
to do:
get from main page if the procedure has some additional error messages.. (прекратена на базата на...)

error:
PdfReadError: Invalid Elementary Object starting with b';' @13776

"""



#additional functions
def iserror(value,dict_):
    try:
        output = dict_[value]
    except KeyError:
        output = np.nan
    return output

def drop_empty_files(path):
    # drop empty files
    empty_files = [os.path.join(os.getcwd(),path,file) for file in os.listdir(os.path.join(os.getcwd(),path)) 
                   if '.json' in file 
                   and os.path.getsize(os.path.join(os.getcwd(),path,file)) ==0]
    
    for file in empty_files:
        os.remove(file)
    
def merge_files(path,delete_prior = False):
    logging.info(f'Start of procedure to merge UCDP export at [{path.upper()}], delete_prior = {delete_prior}')

    #delete previously merged files
    if delete_prior is True:
        merged_files = [os.path.join(os.getcwd(),path,file)
                  for file in os.listdir(os.path.join(os.getcwd(),path)) 
                  if 'merged_' in file and '.json' in file]
        
        for file in merged_files:
            os.remove(file)
        
        logging.info(f'#merged files removed: {len(merged_files)}')
        
    
    # get all relevant files that need to be merged
    list_of_exports = [os.path.join(os.getcwd(),path,file)
                  for file in os.listdir(os.path.join(os.getcwd(),path)) 
                  if 'export_' in file and '.json' in file
                     and os.path.getsize(os.path.join(os.getcwd(),path,file)) !=0
                    ]
    if len(list_of_exports) == 0:
        logging.info(f'No files to merge')
    else:
        logging.info(f'Merging all previously downloaded files: #{len(list_of_exports)}')

        merged_set = []
        for filename in list_of_exports:
            with open(filename ,encoding='ascii') as f:
                data = json.load(f)
                for file in data:
                    if file not in merged_set:
                        merged_set.append(file)

        new_filename = f'merged_{GLOBAL_NAME}_{datetime.now()}.json'
        with open(os.path.join(os.getcwd(),path,new_filename),'w') as f:
            json.dump(merged_set, f)

        logging.info(f'Updated merged file (#{len(merged_set)}) created at {path}: {new_filename} ')




def extract_competitors(url, path):
    
    
    #download pdf file
    response = requests.get(url)
    filename = url.split('/')[-1]
    
    if '.pdf' in filename:
        pass
    else:
        return ['ERROR No pdf file provided']
    
    with open(f"{os.path.join(os.getcwd(),path,filename)}", 'wb') as pdf:
        pdf.write(response.content)
        
    #extract informtion from downloaded file
    with open(f'{os.path.join(os.getcwd(),path,filename)}','rb') as pdfFileObj:
        pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
        
        mentioned_requests = np.nan
        set_requests = []
        
        #checking first 8 pages
        num_pages = pdfReader.numPages
        for page in range(0, np.min([num_pages,8])):
            pageObj = pdfReader.getPage(page)
            
            try:
                text = pageObj.extractText()
                text = text.replace('Заявка с Вх. №','Заявка с Вх №')
                text = text.replace('както следва:','както следва.')

                text_filtered = text.replace(' ','').replace('\n','')
                if 'неерегистрираннитоединучастник' in text_filtered and 'ЗаявкасВх№' not in text:
#                     set_requests.append('не е регистриран нито един участник')   
                    pass
                else:
                    for sent in sent_tokenize(text):
                        if 'заучастиесапостъпили' in sent.replace(' ','') and 'броя' in sent:
                            mentioned_requests = int(re.findall(r'\d+ броя', sent)[0].split('броя')[0])
                        if 'Заявка с Вх №'in sent:
                            request = sent.split('депозирана от ')[-1]
                            set_requests.append(request)
            except:
                print(f'Protocol: {filename} ..... Error with text on page: {page+1}')
                continue

    return [mentioned_requests,set_requests,num_pages]

## spider Class
class WoodSpider(scrapy.Spider):

    name = 'spider'
    allowed_domains = ['auction.ucdp-smolian.com']
    start_urls = ['https://auction.ucdp-smolian.com/publicInfo?view=current']
    base_url = 'https://auction.ucdp-smolian.com'
    
    custom_settings = {
        'FEEDS': {
            f'exports/archive/export_ucdp-smolian_{datetime.now()}.json': {
                'format': 'json',
                'overwrite': True
            }
        }
    }
    
    
    def parse(self, response):
        if len(selected_files)>0:
            logging.info(f'Extracting files from a selected set: {len(selected_files)}')
            all_auctions = selected_files.copy()

            for auction_url in all_auctions:
                if auction_url not in extracted_urls:
                    logging.info(f'Extracting........: {auction_url}')
                    try:
                        yield scrapy.Request(auction_url, callback=self.parse_auction)
                    except:
                        print('issue at download')
                        yield {'url':'auction_url','ISSUE at download:':1}

                else:
                    continue
        else:            
            all_auctions = response.xpath('//tr/td/a')#.extract()
            for auction in all_auctions:
                auction_url = auction.xpath('.//@href').extract_first()
                auction_url = self.base_url + auction_url

                if auction_url not in extracted_urls:
                    print('Request:',auction_url)
                    logging.info(f'Extracting........: {auction_url}')
                    try:
                        yield scrapy.Request(auction_url, callback=self.parse_auction)
                    except:
                        yield {'url':'auction_url','ISSUE at download:':1}

                else:
                    continue
        
            if crawl_all == True:
                string = "//tr/td/ul[@class='pagination']/li/a[contains(concat(' ', i/@class, ' '), ' fa fa-chevron-right ')]/@href"
                next_page_partial_url = response.xpath(string).extract_first()

                if next_page_partial_url:
                    next_page_url = self.base_url + next_page_partial_url            
                    yield scrapy.Request(next_page_url, callback=self.parse)
    
    def parse_auction(self,response):
        print('Parse auction....', response)
        all_content = response.xpath('//tr')
        _dict = {}
        
        _dict['#urls'] = len(all_content.xpath('.//td/a/@href').extract())
        
        for i,content in enumerate(all_content):
            
            ##check if there are links 
            if len(content.xpath('.//td/a/@href'))!=0:
                url = content.xpath('.//td/a/@href').extract()
                
                if len(url)>=1:
                    contents = content.xpath('.//td//text()').extract() #contains titles + data                    
                    value = contents[0]
                    _dict[value.split(' - ')[0]] = self.base_url + url[0]
                    
             #if it does not contain a link       
            else:
                contents = content.xpath('.//td//text()').extract() #contains titles + data
                if len(contents) == 1:
                    value = contents[0]
                    if '\xa0' in value  or 'Резултати' in value or 'Данни за дървесина' in value \
                                    or 'Документация към процедурата' in value:
                        pass
                    
                    elif i == 0:
                        _dict['ДП'] = value
                    elif 'Обект №' in value:
                        _dict['Възложител'] = value.split('/')[0]
                    else:
                        _dict[i] = contents[0]
            
                elif len(contents) == 2:
                        _dict[contents[0]] = contents[1]

                else:
                    new_content = []
                    for value in contents:
                        value = value.replace('\n','').strip()
                        new_content.append(value)
                        new_content = list(filter(None, new_content))

                    for i,value in enumerate(new_content):
                        if 'първо място' in value:
                            _dict['първо място'] = value.split('На първо място')[1].strip()
                        if i == 1:
                            _dict['първа цена'] = value.split('цена в размер на ')[-1]
                        if 'второ място' in value:
                            _dict['второ място'] = value.split('На второ място')[1].strip()
                        if i == 3:
                            _dict['втора цена'] = value.split('цена в размер на ')[-1]
                        if 'трето място' in value:
                            _dict['трето място'] = value.split('На трето място')[1].strip()
                        if i == 5:
                            _dict['трета цена'] = value.split('цена в размер на ')[-1]
        
        #adding some additional fields
        _dict['url'] =  response.url
        
        relevant_urls = [v for k,v in _dict.items() if "протокол" in k.lower()]
        
        if len(relevant_urls)==0:
            pass
        elif len(relevant_urls) > 1:
            print(f'Multiple protocols  for {response.url}')
            _dict['участници (извлечени)'] = 'ERROR, multiple protocols'
            
        else:
            url = relevant_urls[0]
            competitors = extract_competitors(url,path = 'exports/protocols')
            _dict['брой участници (споменати)'] = competitors[0]
            _dict['участници (извлечени)'] = str(competitors[1])
            _dict['брой участници (извлечени)'] = len(competitors[1])
            _dict['страници протокол'] = competitors[2]
                                                   
        fields_mapping = {'url':'url'
                          ,'ДП':'ДП'
                          ,'Възложител':'ДГС/ДЛС'
                          ,'Обект №':'Обект№'
                          ,'№ търг':'Tърг№'
                          ,'Първа дата':'Първа дата'
                          ,'Втора дата':'Втора дата'
                          ,'Предмет':'Предмет'
                          ,'Начална цена':'Начална цена'
                          ,'Дървесен вид':'Дървесен вид'
                          ,'Едра':'Едра'
                          ,'Средна':'Средна'
                          ,'Дребна':'Дребна'
                          ,'ОЗМ':'ОЗМ'
                          ,'Дърва за огрев':'Дърва за огрев'
                          ,'Общо':'Общо'
                          ,"първо място":'Първо място'
                          ,'първа цена':'Първа цена'
                          ,'второ място':'Второ място'
                          ,'втора цена':'Втора цена'
                          ,'брой участници (споменати)':'брой участници (споменати)'
                          ,'участници (извлечени)':'участници (извлечени)'
                          ,'брой участници (извлечени)':'брой участници (извлечени)'
                          ,"Протокол":'Протокол'
                          ,'страници протокол':'страници протокол'
                          ,'Заповед':"Заповед"
                          ,'Заповед за определяне на купувач':'Заповед за определяне на купувач'
                          ,'Заповед за откриване':'Заповед за откриване'
                          ,'Заповед за прекратяване':'Заповед за прекратяване'
                          ,'Договор и приложения':'Договор'
                          ,'Допълнитено споразумение':'Допълнитено споразумение'
                          ,'Изменена документация':'Изменена документация'
                          ,'Заповед за изменение':'Заповед за изменение'
                          ,'Заповед за определяне на изпълнител':'Заповед за определяне на изпълнител'
                          ,'Уведомление за прекратяване на договор':'Уведомление за прекратяване на договор'
                          ,'Приложения':'Приложения'
                          ,'Документация':'Документация'
                          ,'#urls':'#документи'
                         }
        
        mapped = {v:iserror(k,_dict) for k,v in fields_mapping.items()}
        not_mapped = {k:v for k,v in _dict.items() if k not in [i for i in fields_mapping]}
        
        final_dict = {**mapped,**not_mapped} if len(not_mapped) > 0 else mapped
        
        yield final_dict
        
        



In [2]:
path = 'exports/archive'
GLOBAL_NAME = 'UCDP-SMOLIAN'
start_crawl = True
crawl_all = True #tells whether we wnat to crawl all pages or just the start_urls, normally only last page contains new data
# it is not relevant if we are crawling preselected files

#in case we have specific files that we want to download, it checks whether it has been downloaded and downloads it if NOT
selected_files = ['https://auction.ucdp-smolian.com/publicInfo?printid=2321994d85d661d792223f647000c65f'
                  ,'https://auction.ucdp-smolian.com/publicInfo?printid=059fdcd96baeb75112f09fa1dcc740cc' 
                  ,'https://auction.ucdp-smolian.com/publicInfo?printid=bdc4626aa1d1df8e14d80d345b2a442d'
                 ]

selected_files=[]

#Connection lost
# https://auction.ucdp-smolian.com/publicInfo?view=archive&page=420>
# https://auction.ucdp-smolian.com/publicInfo?view=archive&page=640

# tests on validity of data

logging.info('>>>>> Start of UCDP SMOLIAN script')
logging.info(f'>>>>> Crawl UCDP SMOLIAN: {start_crawl} ......')
logging.info(f'>>>>> Crawl all pages: {crawl_all} ......')

if len(selected_files) !=0:
    logging.info(f'>>>>> Total # of pre-selected files to be crawled: {len(selected_files)} ......')


if start_crawl == True:
    logging.info(f'......................SPYDER CRAWLER START......................')

    latest_file, extracted_urls, data = get_latest_file(PATH =path ,type_ ='.json',aux=f'merged_{GLOBAL_NAME}')
    logging.info(f'>>>>> # observations in latest merge: {len(data)} ......')


    process = CrawlerProcess()
    process.crawl(WoodSpider)
    process.start()
    
    logging.info(f'......................SPYDER CRAWLER END......................')
    logging.info(f'>>>>> Finish craw with export path: {path.upper()} ......')
    merge_files(path = path, delete_prior = True)
else:
    pass

# #cleaning part
drop_empty_files(path = path)
latest_file, extracted_urls, data = get_latest_file(PATH=path ,type_ ='.json',aux=f'merged_{GLOBAL_NAME}')
logging.info(f'>>>>> # observations in latest merge: {len(data)} ......')
clean_data = transform_clean_data(data,GLOBAL_NAME )



2023-01-31 11:56:49 [py.warnings] WARNING: /opt/anaconda3/lib/python3.8/site-packages/scrapy/utils/request.py:231: ScrapyDeprecationWarning: '2.6' is a deprecated value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting.

It is also the default value. In other words, it is normal to get this warning if you have not defined a value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting. This is so for backward compatibility reasons, but it will change in a future version of Scrapy.

See the documentation of the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting for information on how to handle this deprecation.
  return cls(crawler)

2023-01-31 11:56:49 [scrapy.extensions.telnet] INFO: Telnet Password: 1b9d05bbd0c35b35
2023-01-31 11:56:49 [scrapy.extensions.logstats] INFO: Crawled 0 pages (at 0 pages/min), scraped 0 items (at 0 items/min)
2023-01-31 11:56:49 [scrapy.extensions.telnet] INFO: Telnet console listening on 127.0.0.1:6023
2023-01-31 11:56:50 [filelock] DEBUG: Attempting to ac

Request: https://auction.ucdp-smolian.com/publicInfo?printid=1cfead9959b76ce44a847c850b61c587
Request: https://auction.ucdp-smolian.com/publicInfo?printid=ffbd6cbb019a1413183c8d08f2929307
Request: https://auction.ucdp-smolian.com/publicInfo?printid=9e6a921fbc428b5638b3986e365d4f21
Request: https://auction.ucdp-smolian.com/publicInfo?printid=87f4d79e36d68c3031ccf6c55e9bbd39
Request: https://auction.ucdp-smolian.com/publicInfo?printid=532b7cbe070a3579f424988a040752f2
Request: https://auction.ucdp-smolian.com/publicInfo?printid=bbb001ba009ed11717eaec9305b2feb6
Request: https://auction.ucdp-smolian.com/publicInfo?printid=366f0bc7bd1d4bf414073cabbadfdfcd
Request: https://auction.ucdp-smolian.com/publicInfo?printid=58191d2a914c6dae66371c9dcdc91b41
Request: https://auction.ucdp-smolian.com/publicInfo?printid=8b313cbf30999888de32da1ec83ff503
Request: https://auction.ucdp-smolian.com/publicInfo?printid=f0031c7a91d74015a9addfbc589f3fe5
Request: https://auction.ucdp-smolian.com/publicInfo?printid

2023-01-31 11:56:54 [root] INFO: Extracting........: https://auction.ucdp-smolian.com/publicInfo?printid=c6a01432c8138d46ba39957a8250e027
2023-01-31 11:56:54 [root] INFO: Extracting........: https://auction.ucdp-smolian.com/publicInfo?printid=c922de9e01cba8a4684f6c3471130e4c
2023-01-31 11:56:54 [root] INFO: Extracting........: https://auction.ucdp-smolian.com/publicInfo?printid=54fda78aa8a09b4d77b5aaec57b75028
2023-01-31 11:56:54 [root] INFO: Extracting........: https://auction.ucdp-smolian.com/publicInfo?printid=fb2606a5068901da92473666256e6e5b
2023-01-31 11:56:54 [root] INFO: Extracting........: https://auction.ucdp-smolian.com/publicInfo?printid=7221e5c8ec6b08ef6d3f9ff3ce6eb1d1
2023-01-31 11:56:54 [root] INFO: Extracting........: https://auction.ucdp-smolian.com/publicInfo?printid=35937e34256cf4e5b2f7da08871d2a0b
2023-01-31 11:56:54 [root] INFO: Extracting........: https://auction.ucdp-smolian.com/publicInfo?printid=8b8388180314a337c9aa3c5aa8e2f37a
2023-01-31 11:56:54 [root] INFO: E

Request: https://auction.ucdp-smolian.com/publicInfo?printid=c6a01432c8138d46ba39957a8250e027
Request: https://auction.ucdp-smolian.com/publicInfo?printid=c922de9e01cba8a4684f6c3471130e4c
Request: https://auction.ucdp-smolian.com/publicInfo?printid=54fda78aa8a09b4d77b5aaec57b75028
Request: https://auction.ucdp-smolian.com/publicInfo?printid=fb2606a5068901da92473666256e6e5b
Request: https://auction.ucdp-smolian.com/publicInfo?printid=7221e5c8ec6b08ef6d3f9ff3ce6eb1d1
Request: https://auction.ucdp-smolian.com/publicInfo?printid=35937e34256cf4e5b2f7da08871d2a0b
Request: https://auction.ucdp-smolian.com/publicInfo?printid=8b8388180314a337c9aa3c5aa8e2f37a
Request: https://auction.ucdp-smolian.com/publicInfo?printid=2ecd2bd94734e5dd392d8678bc64cdab
Request: https://auction.ucdp-smolian.com/publicInfo?printid=8c9a14ffebb7677d033ffce847991293
Request: https://auction.ucdp-smolian.com/publicInfo?printid=86b20716fbd5b253d27cec43127089bc
Request: https://auction.ucdp-smolian.com/publicInfo?printid

2023-01-31 11:56:55 [root] INFO: Extracting........: https://auction.ucdp-smolian.com/publicInfo?printid=32b3ee0272954b956a7d1f86f76afa21
2023-01-31 11:56:55 [root] INFO: Extracting........: https://auction.ucdp-smolian.com/publicInfo?printid=77ee3bc58ce560b86c2b59363281e914
2023-01-31 11:56:55 [root] INFO: Extracting........: https://auction.ucdp-smolian.com/publicInfo?printid=98c39996bf1543e974747a2549b3107c
2023-01-31 11:56:55 [root] INFO: Extracting........: https://auction.ucdp-smolian.com/publicInfo?printid=32cbf687880eb1674a07bf717761dd3a
2023-01-31 11:56:55 [root] INFO: Extracting........: https://auction.ucdp-smolian.com/publicInfo?printid=9aeade7beada35c83d3b344fbafe43b0
2023-01-31 11:56:55 [root] INFO: Extracting........: https://auction.ucdp-smolian.com/publicInfo?printid=fdbd31f2027f20378b1a80125fc862db
2023-01-31 11:56:55 [root] INFO: Extracting........: https://auction.ucdp-smolian.com/publicInfo?printid=b22b257ad0519d4500539da3c8bcf4dd
2023-01-31 11:56:55 [root] INFO: E

...

2023-01-31 11:56:56 [root] INFO: Extracting........: https://auction.ucdp-smolian.com/publicInfo?printid=91378b331327b40e564390c43cd6b2be
2023-01-31 11:56:56 [root] INFO: Extracting........: https://auction.ucdp-smolian.com/publicInfo?printid=c20a7ce2a627ba838cfbff082db35197
2023-01-31 11:56:56 [root] INFO: Extracting........: https://auction.ucdp-smolian.com/publicInfo?printid=f4aa0dd960521e045ae2f20621fb4ee9
2023-01-31 11:56:56 [root] INFO: Extracting........: https://auction.ucdp-smolian.com/publicInfo?printid=8051a3c40561002834e59d566b7430cf
2023-01-31 11:56:56 [root] INFO: Extracting........: https://auction.ucdp-smolian.com/publicInfo?printid=8832ae39136fb470b3fbbf9f47b4570d
2023-01-31 11:56:56 [root] INFO: Extracting........: https://auction.ucdp-smolian.com/publicInfo?printid=07c4b1fdd1ef8da2d7e9393071204b71
2023-01-31 11:56:56 [root] INFO: Extracting........: https://auction.ucdp-smolian.com/publicInfo?printid=f023559c6d59c04e3804209f573dbff9
2023-01-31 11:56:56 [root] INFO: E

Parse auction.... <200 https://auction.ucdp-smolian.com/publicInfo?printid=98c39996bf1543e974747a2549b3107c>
Request: https://auction.ucdp-smolian.com/publicInfo?printid=91378b331327b40e564390c43cd6b2be
Request: https://auction.ucdp-smolian.com/publicInfo?printid=c20a7ce2a627ba838cfbff082db35197
Request: https://auction.ucdp-smolian.com/publicInfo?printid=f4aa0dd960521e045ae2f20621fb4ee9
Request: https://auction.ucdp-smolian.com/publicInfo?printid=8051a3c40561002834e59d566b7430cf
Request: https://auction.ucdp-smolian.com/publicInfo?printid=8832ae39136fb470b3fbbf9f47b4570d
Request: https://auction.ucdp-smolian.com/publicInfo?printid=07c4b1fdd1ef8da2d7e9393071204b71
Request: https://auction.ucdp-smolian.com/publicInfo?printid=f023559c6d59c04e3804209f573dbff9
Request: https://auction.ucdp-smolian.com/publicInfo?printid=766e428d1e232bbdd58664b41346196c
Request: https://auction.ucdp-smolian.com/publicInfo?printid=68a9750337a418a86fe06c1991a1d64c
Request: https://auction.ucdp-smolian.com/pub

2023-01-31 11:56:57 [scrapy.extensions.feedexport] INFO: Stored json feed (75 items) in: exports/archive/export_ucdp-smolian_2023-01-31 11:56:49.402843.json
2023-01-31 11:56:57 [scrapy.statscollectors] INFO: Dumping Scrapy stats:
{'downloader/request_bytes': 31379,
 'downloader/request_count': 79,
 'downloader/request_method_count/GET': 79,
 'downloader/response_bytes': 888941,
 'downloader/response_count': 79,
 'downloader/response_status_count/200': 79,
 'elapsed_time_seconds': 7.996585,
 'feedexport/success_count/FileFeedStorage': 1,
 'finish_reason': 'finished',
 'finish_time': datetime.datetime(2023, 1, 31, 9, 56, 57, 644748),
 'item_scraped_count': 75,
 'log_count/DEBUG': 2,
 'log_count/INFO': 81,
 'log_count/WARNING': 1,
 'memusage/max': 201809920,
 'memusage/startup': 201809920,
 'request_depth_max': 4,
 'response_received_count': 79,
 'scheduler/dequeued': 79,
 'scheduler/dequeued/memory': 79,
 'scheduler/enqueued': 79,
 'scheduler/enqueued/memory': 79,
 'start_time': datetime

Parse auction.... <200 https://auction.ucdp-smolian.com/publicInfo?printid=c922de9e01cba8a4684f6c3471130e4c>
Parse auction.... <200 https://auction.ucdp-smolian.com/publicInfo?printid=7f01c939761597f9b832694d4c381512>


2023-01-31 11:56:57 [numexpr.utils] INFO: Note: NumExpr detected 10 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2023-01-31 11:56:57 [numexpr.utils] INFO: NumExpr defaulting to 8 threads.


In [4]:
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
import plotly as plotly
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots

@interact

def display_historical(town = clean_data['селище'].unique()
                       , color = ['Начална цена'
                                  , 'Обем дървесина (м3)'
                               ,'разлика от начална цена (лв.)'
                               ,'%от начална цена (лв.)'
                               ,'начална цена лв./м3','брой участници (споменати)']
                      ):
    
    df_base = clean_data[clean_data['селище'] == town].reset_index(drop = True)
    
    for type_ in ['average']:
        
        df_date = df_base[['дата', 'година', 'месец']].groupby([ 'година', 'месец']).max().reset_index()
        df_count = df_base[[ 'година', 'месец','Tърг№']].groupby([ 'година', 'месец']).count().reset_index()

        df_count = df_count.merge(df_date,how = 'right',on = ['година','месец'])

        new_column = f'{color}<br>(средна стойност)' if type_ == 'average' else f'{color} (общо)'

        if type_ == 'average':
            df_color = df_base[[ 'година', 'месец',color]].groupby([ 'година', 'месец']).mean().reset_index().rename(columns = {color:new_column})
        else:
            df_color = df_base[[ 'година', 'месец',color]].groupby([ 'година', 'месец']).sum().reset_index().rename(columns = {color:new_column})

        df_value = df_count.merge(df_color,how = 'right',on = ['година','месец'])
        
        fig = make_subplots(specs=[[{"secondary_y": True}]])
        
        fig.add_trace(go.Bar(y = df_value[new_column]
                                 , x = df_value["дата"]
                                 , name = new_column
#                                  ,color = new_column
#                                  ,marker={'color': colors[label]}
                                 ,showlegend = True
                                 ,marker_color = 'blue'
                                 , opacity = 0.6)
                  ,secondary_y = False)
        
        fig.add_trace(go.Scatter(y = df_value['Tърг№']
                                 , x = df_value["дата"]
                                 , name = 'брой поръчки'
                                 ,showlegend = True
                                 ,marker_color = 'red'
                                 ,mode='lines'
                                 ,line=dict(width=2,dash='dot')
                                 , opacity = 1
                                )
                  ,secondary_y = True)

#         fig = px.bar(df_value, x="дата"
#                        ,y = 'Tърг№'
#                        ,color = new_column
#                        ,barmode="group"
#               )

    fig.update_xaxes(type = 'category',showgrid=False)
    fig.update_yaxes(showgrid=False,secondary_y = True, title = 'брой поръчки')


    fig.update_layout(title_text = F'{town}: брой поръчки и {new_column.lower()}'
                      ,template = 'presentation'
                      ,font_size = 15
                        ,height = 500
                        ,width = 1000
#                         , font = {'size':13}
                        ,font=dict(family="Courier New, monospace"
                               ,size=13
                               ,color="black")
                        , xaxis = {"title": ''}
                        , yaxis = {"title": new_column}, legend = {'orientation':"h"
                                                                        ,'yanchor':"middle"
                                                                        ,'y':-0.5
#                                                                         , 'x':0.1
                                                                        ,'title':''}
                     ) 
    
    heatmap_df = clean_data[['селище','година','месец',color]].groupby(['селище','година','месец']).mean().reset_index()
    heatmap_df = heatmap_df.sort_values(['година','месец'])
    heatmap_df['дата'] = heatmap_df['година'].apply(lambda x:str(x)) \
                +'-'+ heatmap_df['месец'].apply(lambda x:str(x) if len(str(x)) == 2 else f'0{x}')

    list_towns = sorted(heatmap_df['селище'].unique())
    list_date = [str(i) for i in heatmap_df['дата'].unique()]

    heatmap_df.set_index(['селище','дата'],inplace = True)

    z = []
    for town in list_towns:
        town_list = []
        for _date in list_date:
            try:
                town_list.append(heatmap_df[color][town,_date])
            except KeyError:
                town_list.append(np.nan)
        z.append(town_list)


    fig_heatmap = px.imshow(z, x=list_date, y=list_towns, color_continuous_scale='YlGnBu', aspect="auto")
#     fig_heatmap.update_traces(text=z)
    fig_heatmap.update_xaxes(side="top")
    fig_heatmap.update_layout(title = f'Карта {color.upper()}(средна стойност) по селище/година-месец'
                    ,template = 'presentation'
                    ,font_size = 15
                    ,height = 1800
                    ,width = 900
                    ,font=dict(family="Courier New, monospace"
                               ,size=11
                               ,color="black")
                            )

    return fig.show(),fig_heatmap.show()
    



interactive(children=(Dropdown(description='town', options=('Селище', 'Ардино', 'Кирково', 'Златоград', 'Славе…

In [5]:

def create_heatmap(data_frame, variable,path = ''):

    heatmap_df = data_frame[['селище','година','месец',variable]].groupby(['селище','година','месец']).mean().reset_index()
    heatmap_df = heatmap_df.sort_values(['година','месец'])
    heatmap_df['дата'] = heatmap_df['година'].apply(lambda x:str(x)) \
                     +'-'+ heatmap_df['месец'].apply(lambda x:str(x) if len(str(x)) == 2 else f'0{x}')

    list_towns = sorted(heatmap_df['селище'].unique())
    list_date = [str(i) for i in heatmap_df['дата'].unique()]

    heatmap_df.set_index(['селище', 'дата'], inplace=True)

    z = []
    for town in list_towns:
        town_list = []
        for _date in list_date:
            try:
                town_list.append(heatmap_df[variable][town, _date])
            except KeyError:
                town_list.append(np.nan)
        z.append(town_list)


    fig_heatmap = px.imshow(z, x=list_date, y=list_towns, color_continuous_scale='YlGnBu', aspect="auto")
    fig_heatmap.update_xaxes(side="top")
    fig_heatmap.update_layout(title = f'Карта {variable.upper()}(средна стойност) по селище/година-месец'
                          ,template = 'presentation'
                          ,font_size = 15
                          ,height = 1800
                          ,width = 900
                          ,font=dict(family="Courier New, monospace"
                                     ,size=11
                                     ,color="black")
                          )
    
    fig_name = f'htmap_{variable[:2]}.png'
    fig_heatmap.write_image(os.path.join("/Users/bilyanavencheva/IdeaProjects/WoodAuctionExtraction/exports/images",fig_name)
                            ,engine = 'auto'
                            ,format='png'
                            ,scale = 3)

    return fig_heatmap


In [ ]:
path = '/Users/bilyanavencheva/IdeaProjects/WoodAuctionExtraction/exports/images'

In [ ]:
import plotly.io as pio
for variable in ['Начална цена',
       'разлика от начална цена (лв.)', '%от начална цена (лв.)',
       'начална цена лв./м3', 'крайна цена лв./м3', 'Договорена цена (лв.)','Втора цена', 
       'брой участници (споменати)',  'Едра', 'Средна',
       'Дребна', 'ОЗМ', 'Дърва за огрев', 'Обем дървесина (м3)']:
    create_heatmap(data_frame = clean_data, variable = variable)

In [ ]:
base_url = 'https://auction.ucdp-smolian.com/files/'
pdf = '0c10c94ae8c968229b2e2e9d7b98a40b.pdf' #'0032f7a2b7d4ba2ad25d93f284e3ea5b.pdf'
url = base_url + pdf

    #download pdf file
response = requests.get(url)
filename = url.split('/')[-1]


with open(f"{os.path.join(os.getcwd(),path,filename)}", 'wb') as pdf:
    pdf.write(response.content)

#extract informtion from downloaded file
with open(f'{os.path.join(os.getcwd(),path,filename)}','rb') as pdfFileObj:
    print(pdfFileObj)
    pdfReader = PyPDF2.PdfReader(pdfFileObj)

    mentioned_requests = np.nan
    set_requests = []

    #checking first pages or all pages?
    num_pages = pdfReader.numPages
    print(num_pages)
    for page in range(0, np.min([num_pages,8])):
        pageObj = pdfReader.getPage(page)

        try:
            text = pageObj.extractText()
            text = text.replace('нито единучастник','нито един участник')
            text = text.replace('Заявка с Вх. №','Заявка с Вх №')
            text = text.replace('както следва:','както следва.')

            text_filtered = text.replace(' ','').replace('\n','')
            if 'неерегистрираннитоединучастник' in text_filtered and 'ЗаявкасВх№' not in text:
                set_requests.append('не е регистриран нито един участник')   
            else:
                for sent in sent_tokenize(text):
                    if 'заучастиесапостъпили' in sent.replace(' ','') and 'броя' in sent:
                        mentioned_requests = int(re.findall(r'\d+ броя', sent)[0].split('броя')[0])
                    if 'Заявка с Вх №'in sent:
                        request = sent.split('депозирана от ')[-1]
                        set_requests.append(request)
        except:
            print(f'Protocol: {filename} ..... Error with text on page: {page+1}')
            continue
            
set_requests
mentioned_requests

In [ ]:
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from io import StringIO

base_url = 'https://auction.ucdp-smolian.com/files/'
filename = '0032f7a2b7d4ba2ad25d93f284e3ea5b.pdf'

def convert_pdf_to_txt(path):
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    codec = 'utf-8'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, laparams=laparams)
    fp = open(path, 'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 3
    caching = True
    pagenos=set()

    for page in PDFPage.get_pages(fp, pagenos
                                  , maxpages=maxpages
                                  , password=password
                                  , caching=caching
                                  , check_extractable=True):
        try:
            interpreter.process_page(page)
        except:
            continue

    text = retstr.getvalue()

    fp.close()
    device.close()
    retstr.close()
    return text

text = convert_pdf_to_txt(f'{os.path.join(os.getcwd(),path,filename)}')
text = text.replace('Заявка с Вх. №','Заявка с Вх №')
text = text.replace('както следва:','както следва.')
for sent in sent_tokenize(text):
    if 'Заявка с Вх №' in sent:
        print(sent)

In [ ]:
index = []
value = '0032f7a2b7d4ba2ad25d93f284e3ea5b.pdf' #'2101'
column = 'Протокол' #'Обект№'
for i in data_frame.index:
    if value in str(data_frame[column][i]):
        index.append(i)
        
data_frame[data_frame.index.isin(index)][['url', 'ДП', 'Възложител', 'Обект№', 'Tърг№', 'Първа дата',
       'Втора дата', 'Предмет', 'Начална цена', 'Дървесен вид', 'Едра',
       'Средна', 'Дребна', 'ОЗМ', 'Дърва за огрев', 'Общо', 'Първо място',
       'Първа цена', 'Второ място', 'Втора цена', 'брой участници (споменати)',
       'участници (извлечени)', 'брой участници (извлечени)']]

In [ ]:
data_frame_old[~data_frame_old.index.isin(data_frame.index)]['url'][939]

In [ ]:
import json
import pandas as pd
filename = 'export_ucdp-smolian_2022-11-07.json'
with open(filename ,encoding='ascii') as f:
    data = json.load(f)
data_frame = pd.DataFrame.from_dict(data)
data_frame.head()



In [ ]:
import json
import pandas as pd
filename = 'export_ucdp-smolian_2022-11-06.json'
with open(filename ,encoding='ascii') as f:
    data = json.load(f)
data_frame_old = pd.DataFrame.from_dict(data)
data_frame_old.head()